In [1]:
import numpy as np
import os
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
%matplotlib qt5
plt.rc('font', size=16)

### CONSTANTS ###
m_u = 1.66054e-24 # unit mass (g)
a0 = 0.5291772109 # Bohr radius (Å)

# SIMULATING A BULK WATER SYSTEM USING CAR-PARRINELLO MOLECULAR DYNAMICS

<div style="text-align: center"><i>Alessandro Cuoghi - April 2023</i></div>

--- 

# *I* - INTRODUCTION
In studying the physics of complex systems, numerical simulations provide an essential tool to determine the values of quantities that are difficult to obtain through experimental techniques, as well as to verify theories or to examine how certain principles affect the dynamics of the system. From this perspective, studies performed via numerical simulations allow also to examine unphysical processes.

Classical molecular dynamics (MD) is a commonly used computational tool for simulating the properties of liquids, solids, and molecules. Each of the atoms or molecules in the simulation is treated as a point mass and Newton’s equations are integrated to compute their motion. In classical MD the forces are derived from empirical laws and the accuracy of the results strongly depends on the chosen model. This approach has enjoyed tremendous success in the treatment of systems ranging from simple liquids and solids to polymers and biological systems such as proteins and nucleic acids. Despite their success, force fields have a number of serious limitations, the major being that, first, these calculations convey no information about electronic properties and, second, force fields generally assume a pre-specified connectivity among the atoms and, therefore, suffer from an inability to describe chemical bond breaking and forming events. On the other hand, density functional theory (DFT) calculations have provided an accurate, albeit approximate, description of the chemical bond in a large variety of systems, altough they are very demanding computationally and it is only realistically possible to do small simulations for short times. This limits the application of DFT schemes to the study of very large and/or disordered systems and to the computation of interatomic forces for MD simulations.

# *II* - AB INITIO MOLECULAR DYNAMICS
The techniques of every MD scheme face with two differents problems:
- how to compute the trajectory by numerically integrating the classical equation of motion for every particle in the system.
- how to define a force field that correctly describes the physics of the system.
 
## *II.I* - INTEGRATING THE EQUATION OF MOTION. THE VERLET ALGORITHM
In order to evolve the position and the velocity of a particle and computen the trajectory, time is *discretized* in many steps of equal length $\Delta t$. Then, the position of every particle $I$ at a successive step $\vec R_I(t+\Delta t)$ depends on the position, velocity and force at the previous time step $t$

$$\vec R_I(t+\Delta t) = \vec R_I(t) + \vec v_I(t)\Delta t + \frac{\vec F_I(t)}{2m_I}\Delta t^2 + \mathcal{O}(\Delta t^3)$$

There exist many algorithms that allow to integrate Newton's equation of motion; one of the most used and the one that is implemented in the simulations for this report is the Verlet's algorithm [$^{[1]}$](#bib-ver) 

<a id='eq-ver'></a>
$$\vec R_I(t+\Delta t) = 2\vec R_I(t) - \vec R_I(t-\Delta t) + \frac{\vec F_I(t)}{2m_I}\Delta t^2 \tag{1}$$

for which is possible to demonstrate that has an error of $\mathcal{O}(\Delta t^4)$. As we want an efficient simulation that allows to compute quantities of interest with precise values, we want to introduce a numerical error that is as small as possible. Thus, it is crucial the choice of the timestep $\Delta t$: it should be very small so that to minimize the error of the integration procedure, but on the other hand, it should be big enough to allow the simulation to run for a given amount of time needed to simulate properly the physics we are interested in within the computational resources available. It turns out that for a Verlet scheme [(Eq. 1)](#eq-ver), a good choice for the timestep is to take the fastest characteristic time of the system and divide it by 30, $\Delta t \approx \tau/30$.

A general scheme for a MD simulation is composed of 4 steps:
1. *Initialize positions and velocities for every particle of the system*. The velocities are found straightforward by sampling the Boltzmann's distribution at a given temperature; the atomic positions must be carefully determined and chosen such that the system is in a relaxed configuration. Hence, most of the time it is a good idea to run a preliminary simulation with a classical force field to get a more or less thermalized initial state to start with. 

2. *Compute the forces and integrate the equation of motion*. As already discussed, this is done by means of the Verlet algorithm [(Eq. 1)](#eq-ver).

3. *Equilibrate the system*, i.e. allow it to reach thermal equilibrium. Altough it is really difficult to let the system staibilize at a *chosen target temperature*; this has to be with the fact that Newton's equation preserves the total energy of the system and thus the simulation is run in what is called the *microcanonical ensemble*. Temperature is an output of the system, not an input. In order to thermalize the system at a target temperature we need to move to the *canonical ensemble*. This is achieved by applying the Nosé–Hoover thermostat[$^{[2]}$](#bib-hoo), so that the system equilibrate at a target temperature. Care must be taken, since the thermostat changes the equation of motion; thus, in order to get meaningful results from the simulation, after the equilibration one must switch off the thermostat and go back to classical equation of motion. Hopefully, the temperature will then remain very close to the target one. 

4. *Compute the physical quantities of interest*. In MD the values of the physical quantities are extracted by performing averages over time and over the ensemble of particles. 

## *II.II* - COMPUTING THE FORCE FIELD. CAR-PARRINELLO MOLECULAR DYNAMICS
We recall from [(Eq. 1)](#eq-ver) that in order to compute the trajectory we need to evaluate at each timestep the forces acting on every particles. They are computed as the derivative of the total energy with respect to the coordinates,

<a id='eq-for'></a>
$$F_I = -\frac{\partial E(\{R\})}{\partial R_I} \tag{2}$$

In classical MD, the energy is computed as a sum of empirical potentials, that usually comprise for example a two-body pair potential (could be the Lennard-Jones potential) with a 3-body correction term and a mean-field term proportional to the density. The more the terms included, the more accurate the model is. 

**Ab initio Molecular Dynamics** consists of techniques that compute the force field *ab initio* starting from the electronic configurations around the nuclei. At every step, *Schroedinger equation* must be solved in order to find the electronic wavefunction and the energy of the system. Then, by means of the *Hellmann-Feynman theorem* we can compute its derivative in an efficient way to evaluate the forces [(Eq. 2)](#eq-for), so that we can evolve the ions coordinates. Nuclei are still considered as classical particles and obey Newton's equation, but in order to compute the force we need to solve quantum mechanical problem. *Born-Oppenheimer Molecular Dynamics* (BOMD) assumes that electrons stay in their ground state and that the electronic configuration only depends on the current atomic configurations (*adiabaticity assumption*), that is, we neglect every contributions to the dynamics that comes from "the history" of the electronic wavefunction.

Solving at each step the Schroedinger equation and then evolve the atomic positions is a huge and expensive computational task and hence ab-initio MD can simulate smaller systems for a reduced amount of time with respect to classical MD. However, since at each timestep the displacement of the nuclei is very small we can expect that the electronic wavefunction does not change that much every timestep and indeed we can take advantage of this fact to evolve the wavefunction. It is not necessary to solve the quantum mechanical probelm from scrtach every time. This is the fundamental assumption of *Car-Parrinello Molecular Dynamics* (CPMD)[$^{[3]}$](#bib-cpmd). We can calculate ”forces” over each point in the space of the wavefunctions and treat them as a classical fluid that is running after a potential that changes at each step, and propagate everything with the Verlet algorithm. It is sort of dynamical simulated annealing, as explained in the original paper. The "classical" Lagrangian used for "evolve" the wavefunction is

<a id='eq-cpla'></a>
$$\mathcal{L} = \sum_\nu \frac{\mu}{2}\int_\Omega d^3r \ |\dot\psi_\nu|^2 + \sum_I \frac{m_I}{2}\dot R_I^2 - E[\{\psi\}, \{R\}] + \sum_{ij}\Lambda_{ij}\left(\int_\Omega d^3r \ \psi_i^*\psi_j-\delta_{ij}\right) \tag{3}$$

where $\mu$ is the ”fake” mass of the electrons, $E$ is the energy functional and the last term represents the orthonormality constraints that keeps the electronic orbitals orthogonal to each other, $\Lambda_{ij}$ being the Lagrangian multipliers. This is in the end a classical Lagrangian, from which we can derive classical equation of motion using Euler-Lagrange equation

<a id='eq-eul'></a>
$$\frac{\partial \mathcal{L}}{\partial x_i} = \frac{d}{dt}\frac{\partial \mathcal{L}}{\partial \dot x_i} \tag{4}$$

Note here that the coordinates that we are using to derive the dynamics are the electron wavefuntions coordinates (one coordinate for the value and one for the time derivative for each point in the discretized space) and the atomic coordinates (usual velocity and positions), as well as the Lagrangian multipliers. The great improvement introduced by CPMD is now clear: the electronic wavefunctions and the atomic coordinates are evolved **simultaneously**, obeying the equations of motion

<a id='eq-cpmd'></a>
$$
\begin{align}
\mu \ddot\psi_\nu(r,t) &= -\frac{\delta E}{\delta \psi^*_\nu} + \sum_{j}\Lambda_{\nu j}\psi_j \tag{5.a}\\
M_I \ddot {\vec R_I} &= -\frac{\partial E}{\partial R_I} \tag{5.b}
\end{align}
$$

Both will be integrated using Verlet scheme [(Eq. 1)](#eq-ver). The first equation of motion [(Eq. 5.a)](#eq-cpmd) tells us that the electronic wavefunction oscillates around the ground state, following "adiabatically" the true minimum of the energy, so that BOMD and CPMD represents "on average" the same dynamics.

# *III* - CODE

In [2]:
work_dir = os.getcwd() + '/'

pack_dir = work_dir + 'packmol/' # packmol input directory
input_dir = work_dir + 'Input/' # input directory
out_dir = work_dir + 'Output/' # output directory
for wk_dir in [pack_dir, input_dir, out_dir]:
    try:
        os.mkdir(wk_dir)
    except:
        _ = None
        
pseudo_dir = work_dir + 'pseudo/' # pseudo directory
        
### SYSTEM ###
m_H = 1.0079 # u
m_O = 16 # u

N_H2O = 32 # number of H2O molecules
rho = 0.96 # density in g/cm^3

In [5]:
m_H2O = N_H2O*(2*m_H+m_O)*m_u # total mass of the system in g
L = (m_H2O/rho)**(1/3)/1e-8   # length of the side of the cube in Å

xyz_mol = 'water.xyz'     # input file with the coordinates of a single water molecule in Å
xyz_box = 'water_box.xyz' # output file with the coordinates of all water molecules in Å
pk_inp = 'water_box.inp'  # packmol input file
pk_tol = 2.               # tolerance

def gen_xyz_mol(filename): # generate the `xyz_mol` file
    # SPC water model
    theta = 104.5 #°
    d = 1 # Å

    ## Coordinates of the water molecule.
    r_O = np.zeros(3) + [1,1,1]
    r_H1 = r_O + [d*np.cos(theta/2*np.pi/180), d*np.sin(theta/2*np.pi/180), 0]
    r_H2 = r_O + [d*np.cos(theta/2*np.pi/180), -d*np.sin(theta/2*np.pi/180), 0]
    
    with open(filename, 'w') as f:
        print('3\n', file=f)
        print(f'O    {r_O[0]:.4f}    {r_O[1]:.4f}    {r_O[2]:.4f}', file=f)
        print(f'H    {r_H1[0]:.4f}    {r_H1[1]:.4f}    {r_H1[2]:.4f}', file=f)
        print(f'H    {r_H2[0]:.4f}    {r_H2[1]:.4f}    {r_H2[2]:.4f}', file=f)

def gen_pk_inp(filename):
    with open(pack_dir+pk_inp, 'w') as f:
        print(f'tolerance {pk_tol:.2f}', file=f)
        print('filetype xyz', file=f)
        print(f'output {xyz_box}', file=f)
        print('', file=f)
        print(f'structure {xyz_mol}', file=f)
        print(f'  number {N_H2O}', file=f)
        print(f'inside cube 0. 0. 0. {L-pk_tol:.2f}', file=f)
        print('end structure', file=f)
  
def gen_xyz_box():
    gen_xyz_mol(pack_dir+xyz_mol)
    gen_pk_inp(pack_dir+pk_inp)  
    
    #pack_cmd = '$HOME/Programs/packmol-20.14.0/bin/packmol'
    pack_cmd = '$HOME/ACuoghi/QE/packmol/packmol'
    os.system(f'cd {pack_dir} && {pack_cmd} < {pk_inp} > packmol.out')
    
def read_init_pos():
    with open(pack_dir+xyz_box, 'r') as f:
        init_pos = f.readline()
        f.readline()
        init_pos += f'Lattice="{L:.2f} 0.0 0.0 0.0 {L:.2f} 0.0 0.0 0.0 {L:.2f}"\n'
        init_pos += f.read()
    return init_pos

comments = {'CONTROL' : 
                {'title' : 'title of the run',
                 'calculation' : 'type of calculation to be performed',
                 'isave' : 'frequency for writing the restart file',
                 'restart_mode' : 'how to start the simulation',
                 'nstep' : 'number of Car-Parrinello steps performed in this run',
                 'iprint' : 'frequency for writing the relevant quantities',
                 'tprnfor' : 'print forces',
                 'dt' : 'timestep in atomic units',
                 'outdir' : 'output directory',
                 'prefix' : 'prepend the prefix to input/output filenames and restart folders',
                 'ndr' : 'number directory read',
                 'ndw' : 'number directory write',
                 'etot_conv_thr' : 'convergence threshold on total energy for ionic minimization',
                 'forc_conv_thr' : 'convergence threshold on forces for ionic minimization',
                 'ekin_conv_thr' : 'convergence criterion for electron minimization',
                 'pseudo_dir' : 'pseudopotential directory'},
           
           'SYSTEM' : 
                {'ibrav' : 'Bravais lattice',
                 'celldm(1)' : "'a' crystallographic constant in a.u.",
                 'nat' : 'number of atoms',
                 'ntyp' : 'type of atoms',
                 'ecutwfc' : 'kinetic energy cutoff (Ry) for wavefunctions'},
           
            'ELECTRONS' : 
                {'electron_dynamics' : 'set how electrons are moving',
                 'emass' : 'effective electron mass in the CP Lagrangian in a.u.',
                 'electron_velocities' : 'set the electron velocities'},
           
            'IONS' : 
                {'ion_dynamics' : 'set how ions are moving',
                 'ion_velocities' : 'set the ionic velocities',
                 'ion_temperature' : 'set how to control the temperature of the ions',
                 'tempw' : 'ionic temperature in K',
                 'fnosep' : 'oscillation frequency of the Nosé–Hoover thermostat (in THz)',
                 'tolp' : 'old timestep of the restart run',
                 'nhpcl' : 'number of thermostats in the Nosé–Hoover chain'}}

def make_input_file(filename, com_space=40, **kwargs):
    inp_file = ""
    for nml in kwargs:
        inp_file += f"&{nml}\n"
        for key,val in kwargs[nml].items():
            inp_str = f"  {key} = {val},"
            inp_file += inp_str + ' '*(com_space-len(inp_str)) + f" ! {comments[nml][key]}\n"
        inp_file += '/ \n\n'
    
    inp_file += "ATOMIC_SPECIES\n" +\
               f"  O  {m_O:.3f}d0  O_ONCV_PBE-1.2.upf\n" +\
               f"  H  {m_H:.4f}d0  H_ONCV_PBE-1.2.upf\n\n"
    
    init_pos = read_init_pos()
    inp_file += "ATOMIC_POSITIONS (angstrom)\n"
    inp_file += init_pos[init_pos.find('"\n')+2:]
    
    with open(input_dir+filename, 'w') as f:
        f.write(inp_file)
        
    return inp_file

def cp_run(inp_file, n_mpi=1, n_omp=1):
    out_file = inp_file.replace('.in', '.out')
    
    cmd_mpirun = 'mpirun'
    cmd_mpirun = f'{cmd_mpirun} -n {n_mpi} -genv OMP_NUM_THREADS={n_omp} -genv I_MPI_PIN_DOMAIN=omp'
    cmd_cp = 'cp.x'
    
    os.system(f'{cmd_mpirun} {cmd_cp} -in {input_dir+inp_file}')

In order to run a simulation we will usage the `cp.x` package from Quantum Espresso[$^{[4,5]}$](#bib-qe1). We have to specify the parameters of the computation in an input file, which is organized into several namelists, followed by other fields (“cards”) introduced by keywords. The namelists are:
- `&CONTROL`: general variables controlling the run, for example the title, the type of calculation, the path to the needed directories, as well as the number of steps and the timestep of the run.
- `&SYSTEM`: structural information on the system under investigation. The system comprises $32$ water molecules placed in a cubic box of length around $10$ Å so that to match the density $0.96$ g/cm$^3$.
- `&ELECTRONS`: set the electronic variables and the electron dynamics.
- `&IONS`: set ionic variables and dynamics.

After the namelists, you have several fields (“cards”) introduced by keywords with self-explanatory names:
- `ATOMIC_SPECIES`: it is specified the pesudopotentials used for each atom type"
- `ATOMIC_POSITIONS`: the coordinates of the atoms. The initial disposition of the water molecules inside the simulation box is generated by the package *packmol*[$^{[6]}$](#bib-pack)."

In [6]:
gen_xyz_box()

## *III.I* - REACHING THE ELECTRONIC GROUND STATE. BORN-OPPENHEIMER MD

It is not a good idea to start immediately with a CPMD simulation because if you start assigning random velocities to the atoms, you can have some sharp changes in the atomic velocities because they could be not consistent with the current atomic positions, or because two atoms could be too near to each other; this will reflect in little kinks to the nuclei and by consequence little kinks to the electronic degrees of freedom, that will cause a little more kinetic energy to the electronic system (that we want as cold as possible) and a little rise in the electronic temperature. Thus the first run, when starting from scratch, is always an electronic minimization, with fixed ions and cell, to bring the electronic system on the ground state relative to the starting atomic configuration, and we are going to do it by means of the *conjugated gradient* method.

In [7]:
## Produce the BOMD input file
bomd_inp = 'cp.water.bomd.in' # name of the file
print_input_file = True # wheter to print the produced file at the end of the subroutine

inp_key = {"&CONTROL" :
               {"title" : f"'Water {N_H2O} molecules'",
                "calculation" : "'cp'",
                "restart_mode" : "'from_scratch'",
                "ndw" : 50,
                "nstep" : 150,
                "iprint" : 10,
                "isave" : 50,
                "tprnfor" : ".TRUE.",
                "dt" : "20.00d0",
                "etot_conv_thr" : "1.0d-5",
                "ekin_conv_thr" : "1.0d-5",
                "forc_conv_thr" : "1.0d-3",
                "prefix" : 'H2O',
                "pseudo_dir" : f"'{pseudo_dir}'",
                "outdir" : f"'{out_dir}'"},
          
           "&SYSTEM" :
               {"ibrav" : "1",
                "celldm(1)" : f"{L/a0:.2f}",
                "nat" : N_H2O*3,
                "ntyp" : 2,
                "ecutwfc" : "60.00"},
          
           "&ELECTRONS" :
               {"electron_dynamics" : "'cg'"},
          
           "&IONS" : 
               {"ion_dynamics" : "'verlet'",
                "ion_velocities" : "'random'",
                "tempw" : "400.00d0"}}

inp_file = make_input_file(bomd_inp, CONTROL=inp_key["&CONTROL"],  SYSTEM=inp_key["&SYSTEM"],  
                                     ELECTRONS=inp_key["&ELECTRONS"], IONS=inp_key["&IONS"])
    
## Print the input file
if print_input_file==True: print(inp_file, end='')

&CONTROL
  title = 'Water 32 molecules',          ! title of the run
  calculation = 'cp',                    ! type of calculation to be performed
  restart_mode = 'from_scratch',         ! how to start the simulation
  ndw = 50,                              ! number directory write
  nstep = 150,                           ! number of Car-Parrinello steps performed in this run
  iprint = 10,                           ! frequency for writing the relevant quantities
  isave = 50,                            ! frequency for writing the restart file
  tprnfor = .TRUE.,                      ! print forces
  dt = 20.00d0,                          ! timestep in atomic units
  etot_conv_thr = 1.0d-5,                ! convergence threshold on total energy for ionic minimization
  ekin_conv_thr = 1.0d-5,                ! convergence criterion for electron minimization
  forc_conv_thr = 1.0d-3,                ! convergence threshold on forces for ionic minimization
  prefix = H2O,                

In [6]:
## BOMD RUN
cp_run(bomd_inp, n_mpi=3, n_omp=2)


     Program CP v.7.2 starts on 17Apr2023 at 13:15:12 

     This program is part of the open-source Quantum ESPRESSO suite
     for quantum simulation of materials; please cite
         "P. Giannozzi et al., J. Phys.:Condens. Matter 21 395502 (2009);
         "P. Giannozzi et al., J. Phys.:Condens. Matter 29 465901 (2017);
         "P. Giannozzi et al., J. Chem. Phys. 152 154105 (2020);
          URL http://www.quantum-espresso.org", 
     in publications or presentations arising from this work. More details at
     http://www.quantum-espresso.org/quote

     Parallel version (MPI & OpenMP), running on       6 processor cores
     Number of MPI processes:                 3
     Threads/MPI process:                     2

     MPI processes distributed on     1 nodes
     R & G space division:  proc/nbgrp/npool/nimage =       3
     1074 MiB available memory on the printing compute node when the environment starts
 
     Reading input from /home/maglab1/ACuoghi/QE/H2O/Input/cp.water.b


   Cell parameters from input file are used in electron mass preconditioning
   init_tpiba2=    0.11075307
 
   Short Legend and Physical Units in the Output
   ---------------------------------------------
   NFI    [int]          - step index
   EKINC  [HARTREE A.U.] - kinetic energy of the fictitious electronic dynamics
   TEMPH  [K]            - Temperature of the fictitious cell dynamics
   TEMP   [K]            - Ionic temperature
   ETOT   [HARTREE A.U.] - Scf total energy (Kohn-Sham hamiltonian)
   ENTHAL [HARTREE A.U.] - Enthalpy ( ETOT + P * V )
   ECONS  [HARTREE A.U.] - Enthalpy + kinetic energy of ions and cell
   ECONT  [HARTREE A.U.] - Constant of motion for the CP lagrangian
 


   Wave Initialization: random initial wave-functions
   Occupation number from init
   nbnd =   128
    2.00 2.00 2.00 2.00 2.00 2.00 2.00 2.00 2.00 2.00
    2.00 2.00 2.00 2.00 2.00 2.00 2.00 2.00 2.00 2.00
    2.00 2.00 2.00 2.00 2.00 2.00 2.00 2.00 2.00 2.00
    2.00 2.00 2.00 2.00 2.00 2.0

NOTE: eigenvalues are not computed without ortho
 
       nfi   tempp            E -T.S-mu.nbsp         +K_p #Iter
Step    70    1213   -548.889068   -548.889068   -548.335659    11
Step    71    1212   -548.888657   -548.888657   -548.335731     6
Step    72    1210   -548.887857   -548.887857   -548.335817    11
Step    73    1205   -548.885509   -548.885509   -548.335848    11
Step    74    1196   -548.881283   -548.881283   -548.335799    11
Step    75    1184   -548.876013   -548.876013   -548.335709    11
Step    76    1174   -548.871134   -548.871134   -548.335636    11
Step    77    1167   -548.867840   -548.867840   -548.335607    11
Step    78    1164   -548.866655   -548.866655   -548.335612    11
Step    79    1166   -548.867520   -548.867520   -548.335635    11
NOTE: eigenvalues are not computed without ortho
 
       nfi   tempp            E -T.S-mu.nbsp         +K_p #Iter
Step    80    1171   -548.870024   -548.870024   -548.335670    11
Step    81    1179   -548.873541 


   writing restart file (with schema): /home/maglab1/ACuoghi/QE/H2O/Output/H2O_50.save/
     restart      :      0.44s CPU      0.35s WALL (       2 calls)
Step   101    1175   -548.872090   -548.872090   -548.335821     6
Step   102    1187   -548.877292   -548.877292   -548.335824    11
Step   103    1195   -548.880803   -548.880803   -548.335783    11
Step   104    1200   -548.883270   -548.883270   -548.335710    11
Step   105    1206   -548.885829   -548.885829   -548.335645    11
Step   106    1214   -548.889458   -548.889458   -548.335649    11
Step   107    1224   -548.894091   -548.894091   -548.335748    11
Step   108    1233   -548.898260   -548.898260   -548.335897    11
Step   109    1236   -548.899778   -548.899778   -548.336007    11
NOTE: eigenvalues are not computed without ortho
 
       nfi   tempp            E -T.S-mu.nbsp         +K_p #Iter
Step   110    1230   -548.897084   -548.897084   -548.336004    11
Step   111    1215   -548.890346   -548.890346   -548.3358


   writing restart file (with schema): /home/maglab1/ACuoghi/QE/H2O/Output/H2O_50.save/
     restart      :      0.64s CPU      0.51s WALL (       3 calls)


   Averaged Physical Quantities
                      accumulated      this run
   ekinc         :        0.00000       0.00000 (AU)
   ekin          :      396.36641     396.36641 (AU)
   epot          :     -918.46497    -918.46497 (AU)
   total energy  :     -548.81775    -548.81775 (AU)
   temperature   :     1056.98119    1056.98119 (K )
   enthalpy      :        0.00000       0.00000 (AU)
   econs         :     -548.33574    -548.33574 (AU)
   pressure      :        0.00000       0.00000 (Gpa)
   volume        :     6729.85907    6729.85907 (AU)



     Called by MAIN_LOOP:
     initialize   :      3.20s CPU      1.71s WALL (       1 calls)
     main_loop    :  25342.17s CPU  13681.90s WALL (     150 calls)
     cpr_total    :  25343.07s CPU  13682.55s WALL (       1 calls)

     Called by INIT_RUN:

     Called by CPR:
   

0
0
0


0

## *III.II* - CAR-PARRINELLO MD

Now that the system is more or less equilibrated, we can perform the actual CPMD simulation. In the code this is achieved by changing the `electron_dynamics` parameter to `verlet`. Moreover, we have to set a proper value for the electronic 'fake' mass $\mu$ appearing in the CP Lagrangian [(Eq. 3)](#eq-cpla). The fake electron mass is the parameter that change how the wavefunctions follow the minimum of the DFT energy during the dynamics. The smaller the better. In the limit where both the fake mass and the timestep are zero we recover a perfect minimization of the wavefunction at each timestep, but this would require an infinite amount of steps to simulate a trajectory of a given length in time. So we need a compromise between efficiency and precision. It should not be too large either, since we want the electrons to follow adiabatically the ions and, moreover, since they should remain as close as possible to the ground state, we do not want them to couple with the ionic degrees of freedom and exchange energy with them. Furthermore, $\mu$ should not be too small either, otherwise we would need an infinite small timestep to properly follow the electron motion. We set $\mu$ with an "educated guess" of $50$ a.u., and we will check later the output files to see if this choice is reliable enough. With this choice of the effective electronic mass we have to reduce the timestep to $5$ atomic units.

### *III.II.I* - REACHING A GOAL TEMPERATURE - SIMULATING THE CANONICAL ENSEMBLE USING NOSÈ-HOOVER

From the output files we see that the system has a temperature that is very different from the one chosen to set the initial ionic velocities. In order to thermalize the system at a given temperature we need to implement the Nosé–Hoover thermostat in the code and let the system stabilize. This is achieved by setting the parameter `ion_temperature` to `nose`.

In [8]:
## Produce the Nosé–Hoover input file
nose_inp = 'cp.water.nose.in' # name of the file
print_input_file = True # wheter to print the produced file at the end of the subroutine

inp_key = {"&CONTROL" :
               {"title" : f"'Water {N_H2O} molecules'",
                "calculation" : "'cp'",
                "restart_mode" : "'restart'",
                "ndr" : 50,
                "ndw" : 51,
                "nstep" : 100,
                "iprint" : 40,
                "isave" : 50,
                "tprnfor" : ".TRUE.",
                "dt" : "5.00d0",
                "etot_conv_thr" : "1.0d-5",
                "ekin_conv_thr" : "1.0d-5",
                "forc_conv_thr" : "1.0d-3",
                "prefix" : 'H2O',
                "pseudo_dir" : f"'{pseudo_dir}'",
                "outdir" : f"'{out_dir}'"},
          
           "&SYSTEM" :
               {"ibrav" : "1",
                "celldm(1)" : f"{L/a0:.2f}",
                "nat" : N_H2O*3,
                "ntyp" : 2,
                "ecutwfc" : "60.00"},
          
           "&ELECTRONS" :
               {"electron_dynamics" : "'verlet'",
                "emass" : '50.d0',
                "electron_velocities" : "'change_step'"},
          
           "&IONS" : 
               {"ion_dynamics" : "'verlet'",
                "ion_velocities" : "'change_step'",
                "tolp" : "20.d0",
                "ion_temperature" : "'nose'",
                "tempw" : "400.d0",
                "fnosep" : "10.0",
                "nhpcl" : 3}}

inp_file = make_input_file(nose_inp, CONTROL=inp_key["&CONTROL"],  SYSTEM=inp_key["&SYSTEM"],  
                                     ELECTRONS=inp_key["&ELECTRONS"], IONS=inp_key["&IONS"])
    
## Print the input file
if print_input_file==True: print(inp_file, end='')

&CONTROL
  title = 'Water 32 molecules',          ! title of the run
  calculation = 'cp',                    ! type of calculation to be performed
  restart_mode = 'restart',              ! how to start the simulation
  ndr = 50,                              ! number directory read
  ndw = 51,                              ! number directory write
  nstep = 100,                           ! number of Car-Parrinello steps performed in this run
  iprint = 40,                           ! frequency for writing the relevant quantities
  isave = 50,                            ! frequency for writing the restart file
  tprnfor = .TRUE.,                      ! print forces
  dt = 5.00d0,                           ! timestep in atomic units
  etot_conv_thr = 1.0d-5,                ! convergence threshold on total energy for ionic minimization
  ekin_conv_thr = 1.0d-5,                ! convergence criterion for electron minimization
  forc_conv_thr = 1.0d-3,                ! convergence threshold

In [13]:
## NOSÉ–HOOVER THERMOSTAT
cp_run(nose_inp, n_mpi=3, n_omp=2)


     Program CP v.7.2 starts on 17Apr2023 at 17:23: 2 

     This program is part of the open-source Quantum ESPRESSO suite
     for quantum simulation of materials; please cite
         "P. Giannozzi et al., J. Phys.:Condens. Matter 21 395502 (2009);
         "P. Giannozzi et al., J. Phys.:Condens. Matter 29 465901 (2017);
         "P. Giannozzi et al., J. Chem. Phys. 152 154105 (2020);
          URL http://www.quantum-espresso.org", 
     in publications or presentations arising from this work. More details at
     http://www.quantum-espresso.org/quote

     Parallel version (MPI & OpenMP), running on       6 processor cores
     Number of MPI processes:                 3
     Threads/MPI process:                     2

     MPI processes distributed on     1 nodes
     R & G space division:  proc/nbgrp/npool/nimage =       3
     2068 MiB available memory on the printing compute node when the environment starts
 
     Reading input from /home/maglab1/ACuoghi/QE/H2O/Input/cp.water.n

 Electron velocities rescaled with tolp

   formf: eself=   970.22763
   formf:     vps(g=0)=  -0.0001841     rhops(g=0)=  -0.0008854
   formf:     vps(g=0)=  -0.0001841     rhops(g=0)=  -0.0008854
   formf:     vps(g=0)=  -0.0001827     rhops(g=0)=  -0.0008793
   formf:     vps(g=0)=  -0.0001827     rhops(g=0)=  -0.0008793
   formf:     vps(g=0)=  -0.0001827     rhops(g=0)=  -0.0008793
   formf: sum_g vps(g)=  -0.9579407 sum_g rhops(g)=  -4.3105308
   formf:     vps(g=0)=  -0.0000900     rhops(g=0)=  -0.0001476
   formf:     vps(g=0)=  -0.0000900     rhops(g=0)=  -0.0001476
   formf:     vps(g=0)=  -0.0000896     rhops(g=0)=  -0.0001465
   formf:     vps(g=0)=  -0.0000896     rhops(g=0)=  -0.0001465
   formf:     vps(g=0)=  -0.0000896     rhops(g=0)=  -0.0001465
   formf: sum_g vps(g)=  -1.1465899 sum_g rhops(g)=  -0.7184218
   Delta V(G=0):   0.029852Ry,    0.812310eV
 tausm & xnhpm were rescaled 
 
  nfi     ekinc              temph  tempp     etot                 enthal            

    195    0.013851484058667    0.0 1002.74    -548.832414743126    -548.832414743126    -548.375144188833    -548.315251670718   0.0000   0.0000   0.0004   0.0475
    196    0.013792861131088    0.0  998.60    -548.832166215608    -548.832166215608    -548.376782942392    -548.315250339943   0.0000   0.0000   0.0004   0.0494
    197    0.013690093401352    0.0  994.97    -548.832106031585    -548.832106031585    -548.378377406236    -548.315247403033   0.0000   0.0000   0.0004   0.0513
    198    0.013572360954172    0.0  991.83    -548.832254387875    -548.832254387875    -548.379961473946    -548.315248772669   0.0000   0.0000   0.0004   0.0531
    199    0.013483510117524    0.0  989.06    -548.832604742793    -548.832604742793    -548.381571317436    -548.315247992000   0.0000   0.0000   0.0004   0.0550

 * Physical Quantities at step:   200

   from rhoofr: total integrated electronic density
   in g-space =    256.000000   in r-space =   256.000000


                total energy

    201    0.013392932649920    0.0  984.70    -548.833920686515    -548.833920686515    -548.384874988138    -548.315250939378   0.0000   0.0000   0.0004   0.0588
    202    0.013328292247650    0.0  983.20    -548.834859307774    -548.834859307774    -548.386500482315    -548.315251273043   0.0000   0.0000   0.0004   0.0607
    203    0.013255269639200    0.0  982.08    -548.835963678035    -548.835963678035    -548.388113205849    -548.315252280088   0.0000   0.0000   0.0004   0.0625
    204    0.013199110266723    0.0  981.26    -548.837212221998    -548.837212221998    -548.389739173910    -548.315255627569   0.0000   0.0000   0.0004   0.0644
    205    0.013158169835969    0.0  980.65    -548.838566699698    -548.838566699698    -548.391370732298    -548.315256247036   0.0000   0.0000   0.0004   0.0663
    206    0.013125088732017    0.0  980.22    -548.840004263697    -548.840004263697    -548.393003367740    -548.315257914339   0.0000   0.0000   0.0004   0.0682
    207    0.013

0
0
0


### *III.II.II* - REACHING AGAIN THE GROUND STATE
After switching off the thermostat, it is convenient to do a single step of BOMD in order to "cool down" the electrons and minimize their wavefunctions with the conjugated gradient method before starting the actual production run. In this way we are also initializing again the wavefunctions velocities.

In [9]:
## Single step of BOMD
cg_inp = 'cp.water.cg.in' # name of the file
print_input_file = True # wheter to print the produced file at the end of the subroutine

inp_key = {"&CONTROL" :
               {"title" : f"'Water {N_H2O} molecules'",
                "calculation" : "'cp'",
                "restart_mode" : "'restart'",
                "ndr" : 51,
                "ndw" : 52,
                "nstep" : 1,
                "iprint" : 40,
                "isave" : 50,
                "tprnfor" : ".TRUE.",
                "dt" : "5.00d0",
                "etot_conv_thr" : "1.0d-5",
                "ekin_conv_thr" : "1.0d-5",
                "forc_conv_thr" : "1.0d-3",
                "prefix" : 'H2O',
                "pseudo_dir" : f"'{pseudo_dir}'",
                "outdir" : f"'{out_dir}'"},
          
           "&SYSTEM" :
               {"ibrav" : "1",
                "celldm(1)" : f"{L/a0:.2f}",
                "nat" : N_H2O*3,
                "ntyp" : 2,
                "ecutwfc" : "60.00"},
          
           "&ELECTRONS" :
               {"electron_dynamics" : "'cg'"},
          
           "&IONS" : 
               {"ion_dynamics" : "'verlet'"}}

inp_file = make_input_file(cg_inp, CONTROL=inp_key["&CONTROL"],  SYSTEM=inp_key["&SYSTEM"],  
                                   ELECTRONS=inp_key["&ELECTRONS"], IONS=inp_key["&IONS"])
    
## Print the input file
if print_input_file==True: print(inp_file, end='')

&CONTROL
  title = 'Water 32 molecules',          ! title of the run
  calculation = 'cp',                    ! type of calculation to be performed
  restart_mode = 'restart',              ! how to start the simulation
  ndr = 51,                              ! number directory read
  ndw = 52,                              ! number directory write
  nstep = 1,                             ! number of Car-Parrinello steps performed in this run
  iprint = 40,                           ! frequency for writing the relevant quantities
  isave = 50,                            ! frequency for writing the restart file
  tprnfor = .TRUE.,                      ! print forces
  dt = 5.00d0,                           ! timestep in atomic units
  etot_conv_thr = 1.0d-5,                ! convergence threshold on total energy for ionic minimization
  ekin_conv_thr = 1.0d-5,                ! convergence criterion for electron minimization
  forc_conv_thr = 1.0d-3,                ! convergence threshold

In [10]:
## SINGLE STEP BOMD
cp_run(cg_inp, n_mpi=3, n_omp=2)


     Program CP v.7.2 starts on 19Apr2023 at 14: 2:32 

     This program is part of the open-source Quantum ESPRESSO suite
     for quantum simulation of materials; please cite
         "P. Giannozzi et al., J. Phys.:Condens. Matter 21 395502 (2009);
         "P. Giannozzi et al., J. Phys.:Condens. Matter 29 465901 (2017);
         "P. Giannozzi et al., J. Chem. Phys. 152 154105 (2020);
          URL http://www.quantum-espresso.org", 
     in publications or presentations arising from this work. More details at
     http://www.quantum-espresso.org/quote

     Parallel version (MPI & OpenMP), running on       6 processor cores
     Number of MPI processes:                 3
     Threads/MPI process:                     2

     MPI processes distributed on     1 nodes
     R & G space division:  proc/nbgrp/npool/nimage =       3
     2717 MiB available memory on the printing compute node when the environment starts
 
     Reading input from /home/acuoghi/Programmazione/LQSM/Water/QE-wa

 
   Reciprocal Space Mesh
   ---------------------
   Large Mesh
     Global(ngm_g)    MinLocal       MaxLocal      Average
         211251          70417          70417       70417.00
   Smooth Mesh
     Global(ngms_g)   MinLocal       MaxLocal      Average
         211251          70417          70417       70417.00
   Wave function Mesh
     Global(ngw_g)    MinLocal       MaxLocal      Average
          26318           8772           8773        8772.67


   System geometry initialization
   ------------------------------
   cell parameters read from restart file
   ibrav =    1       cell parameters 


   cell at current step : h(t)
      18.88000       0.00000       0.00000
       0.00000      18.88000       0.00000
       0.00000       0.00000      18.88000

   cell at previous step : h(t-dt)
      18.88000       0.00000       0.00000
       0.00000      18.88000       0.00000
       0.00000       0.00000      18.88000
 
     Subspace diagonalization in iterative solution of th


   writing restart file (with schema): /home/acuoghi/Programmazione/LQSM/Water/QE-water/Output/H2O_52.save/
     restart      :      0.30s CPU      0.29s WALL (       1 calls)


   Averaged Physical Quantities
                      accumulated      this run
   ekinc         :        0.00563       0.00000 (AU)
   ekin          :      396.47036     396.75143 (AU)
   epot          :     -918.54206    -918.78378 (AU)
   total energy  :     -549.06497    -549.13932 (AU)
   temperature   :      493.87085     333.65823 (K )
   enthalpy      :     -517.20345       0.00000 (AU)
   econs         :     -548.83975    -548.98717 (AU)
   pressure      :        0.00000       0.00000 (Gpa)
   volume        :     6729.85907    6729.85907 (AU)



     Called by MAIN_LOOP:
     initialize   :      1.68s CPU      1.17s WALL (       1 calls)
     main_loop    :    679.55s CPU    376.76s WALL (       1 calls)
     cpr_total    :    679.87s CPU    377.07s WALL (       1 calls)

     Called by INIT_RUN:
    

0
0
0


### *III.II.III* - NVE SIMULATION
Eventually, we can start our production CPMD simulation that will run for a considerable number of steps. After that we can analyze the trajectories and extract the desired results.

In [13]:
## Produce the Nosé–Hoover input file
nve_inp = 'cp.water.nve.in' # name of the file
print_input_file = True # wheter to print the produced file at the end of the subroutine

inp_key = {"&CONTROL" :
               {"title" : f"'Water {N_H2O} molecules'",
                "calculation" : "'cp'",
                "restart_mode" : "'restart'",
                "ndr" : 52,
                "ndw" : 53,
                "nstep" : 500,
                "iprint" : 40,
                "isave" : 50,
                "tprnfor" : ".TRUE.",
                "dt" : "5.00d0",
                "etot_conv_thr" : "1.0d-5",
                "ekin_conv_thr" : "1.0d-5",
                "forc_conv_thr" : "1.0d-3",
                "prefix" : 'H2O',
                "pseudo_dir" : f"'{pseudo_dir}'",
                "outdir" : f"'{out_dir}'"},
          
           "&SYSTEM" :
               {"ibrav" : "1",
                "celldm(1)" : f"{L/a0:.2f}",
                "nat" : N_H2O*3,
                "ntyp" : 2,
                "ecutwfc" : "60.00"},
          
           "&ELECTRONS" :
               {"electron_dynamics" : "'verlet'",
                "emass" : '50.d0'},
          
           "&IONS" : 
               {"ion_dynamics" : "'verlet'"}}

inp_file = make_input_file(nve_inp, CONTROL=inp_key["&CONTROL"],  SYSTEM=inp_key["&SYSTEM"],  
                                    ELECTRONS=inp_key["&ELECTRONS"], IONS=inp_key["&IONS"])
    
## Print the input file
if print_input_file==True: print(inp_file, end='')

&CONTROL
  title = 'Water 32 molecules',          ! title of the run
  calculation = 'cp',                    ! type of calculation to be performed
  restart_mode = 'restart',              ! how to start the simulation
  ndr = 52,                              ! number directory read
  ndw = 53,                              ! number directory write
  nstep = 500,                           ! number of Car-Parrinello steps performed in this run
  iprint = 40,                           ! frequency for writing the relevant quantities
  isave = 50,                            ! frequency for writing the restart file
  tprnfor = .TRUE.,                      ! print forces
  dt = 5.00d0,                           ! timestep in atomic units
  etot_conv_thr = 1.0d-5,                ! convergence threshold on total energy for ionic minimization
  ekin_conv_thr = 1.0d-5,                ! convergence criterion for electron minimization
  forc_conv_thr = 1.0d-3,                ! convergence threshold

In [14]:
## PRODUCTION RUN
cp_run(nve_inp, n_mpi=1, n_omp=2)


     Program CP v.7.2 starts on 19Apr2023 at 14:12:48 

     This program is part of the open-source Quantum ESPRESSO suite
     for quantum simulation of materials; please cite
         "P. Giannozzi et al., J. Phys.:Condens. Matter 21 395502 (2009);
         "P. Giannozzi et al., J. Phys.:Condens. Matter 29 465901 (2017);
         "P. Giannozzi et al., J. Chem. Phys. 152 154105 (2020);
          URL http://www.quantum-espresso.org", 
     in publications or presentations arising from this work. More details at
     http://www.quantum-espresso.org/quote

     Parallel version (MPI & OpenMP), running on       2 processor cores
     Number of MPI processes:                 1
     Threads/MPI process:                     2

     MPI processes distributed on     1 nodes
     2590 MiB available memory on the printing compute node when the environment starts
 
     Reading input from /home/acuoghi/Programmazione/LQSM/Water/QE-water/Input/cp.water.nve.in

   Job Title: Water 32 molecules




 
   Reciprocal Space Mesh
   ---------------------
   Large Mesh
     Global(ngm_g)    MinLocal       MaxLocal      Average
         211251         211251         211251      211251.00
   Smooth Mesh
     Global(ngms_g)   MinLocal       MaxLocal      Average
         211251         211251         211251      211251.00
   Wave function Mesh
     Global(ngw_g)    MinLocal       MaxLocal      Average
          26318          26318          26318       26318.00


   System geometry initialization
   ------------------------------
   cell parameters read from restart file
   ibrav =    1       cell parameters 


   cell at current step : h(t)
      18.88000       0.00000       0.00000
       0.00000      18.88000       0.00000
       0.00000       0.00000      18.88000

   cell at previous step : h(t-dt)
      18.88000       0.00000       0.00000
       0.00000      18.88000       0.00000
       0.00000       0.00000      18.88000
 
     Subspace diagonalization in iterative solution of th

   2630    0.004574795397102    0.0  358.69    -549.151017057348    -549.151017057348    -548.987446992841    -548.982872197444   0.0000   0.0000   0.0000   0.0000
   2631    0.004549700031097    0.0  358.64    -549.150970312177    -549.150970312177    -548.987421211497    -548.982871511466   0.0000   0.0000   0.0000   0.0000
   2632    0.004536088381756    0.0  358.52    -549.150901510668    -549.150901510668    -548.987407276528    -548.982871188146   0.0000   0.0000   0.0000   0.0000
   2633    0.004541558965145    0.0  358.33    -549.150818728361    -549.150818728361    -548.987412484622    -548.982870925657   0.0000   0.0000   0.0000   0.0000
   2634    0.004554373888518    0.0  358.11    -549.150730030046    -549.150730030046    -548.987424551417    -548.982870177528   0.0000   0.0000   0.0000   0.0000
   2635    0.004560493550967    0.0  357.91    -549.150644999873    -549.150644999873    -548.987430179791    -548.982869686240   0.0000   0.0000   0.0000   0.0000
   2636    0.004

   2679    0.004696881358396    0.0  372.36    -549.157369945427    -549.157369945427    -548.987564600682    -548.982867719324   0.0000   0.0000   0.0000   0.0000
   2680    0.004691303000508    0.0  371.85    -549.157128693093    -549.157128693093    -548.987558001770    -548.982866698770   0.0000   0.0000   0.0000   0.0000
   2681    0.004688890953069    0.0  371.47    -549.156952857870    -549.156952857870    -548.987554545296    -548.982865654343   0.0000   0.0000   0.0000   0.0000
   2682    0.004695293078056    0.0  371.25    -549.156856691611    -549.156856691611    -548.987560150774    -548.982864857696   0.0000   0.0000   0.0000   0.0000
   2683    0.004706371883518    0.0  371.21    -549.156851861748    -549.156851861748    -548.987570570359    -548.982864198476   0.0000   0.0000   0.0000   0.0000
   2684    0.004712300956889    0.0  371.41    -549.156948480535    -549.156948480535    -548.987576197015    -548.982863896058   0.0000   0.0000   0.0000   0.0000
   2685    0.004

   2728    0.004907865209993    0.0  399.23    -549.169830584910    -549.169830584910    -548.987774562129    -548.982866696919   0.0000   0.0000   0.0000   0.0000
   2729    0.004907766269253    0.0  399.31    -549.169869967551    -549.169869967551    -548.987774421554    -548.982866655285   0.0000   0.0000   0.0000   0.0000
   2730    0.004911170425823    0.0  399.50    -549.169959815158    -549.169959815158    -548.987777855380    -548.982866684954   0.0000   0.0000   0.0000   0.0000
   2731    0.004913993725322    0.0  399.80    -549.170099338985    -549.170099338985    -548.987780684479    -548.982866690753   0.0000   0.0000   0.0000   0.0000
   2732    0.004914035097257    0.0  400.21    -549.170287308881    -549.170287308881    -548.987780983863    -548.982866948766   0.0000   0.0000   0.0000   0.0000
   2733    0.004912181110775    0.0  400.73    -549.170520895105    -549.170520895105    -548.987779432063    -548.982867250952   0.0000   0.0000   0.0000   0.0000
   2734    0.004

...

     restart      :      1.95s CPU      1.66s WALL (       7 calls)
   2951    0.004255132149573    0.0  352.00    -549.147646613230    -549.147646613230    -548.987127393569    -548.982872261419   0.0000   0.0000   0.0000   0.0000
   2952    0.004250349474415    0.0  352.24    -549.147753039848    -549.147753039848    -548.987122930603    -548.982872581128   0.0000   0.0000   0.0000   0.0000
   2953    0.004245430153175    0.0  352.43    -549.147835818652    -549.147835818652    -548.987118235650    -548.982872805497   0.0000   0.0000   0.0000   0.0000
   2954    0.004237520273026    0.0  352.57    -549.147891310795    -549.147891310795    -548.987110514648    -548.982872994375   0.0000   0.0000   0.0000   0.0000
   2955    0.004227043301025    0.0  352.65    -549.147916763744    -549.147916763744    -548.987100166051    -548.982873122750   0.0000   0.0000   0.0000   0.0000
   2956    0.004217259465226    0.0  352.66    -549.147910524139    -549.147910524139    -548.987090414884    -5

     restart      :      2.21s CPU      1.87s WALL (       8 calls)
   3001    0.004419103608425    0.0  364.26    -549.153401296819    -549.153401296819    -548.987291540247    -548.982872436638   0.0000   0.0000   0.0000   0.0000
   3002    0.004429704895917    0.0  364.75    -549.153634038867    -549.153634038867    -548.987302327017    -548.982872622121   0.0000   0.0000   0.0000   0.0000
   3003    0.004440199592225    0.0  365.19    -549.153845773140    -549.153845773140    -548.987313028387    -548.982872828795   0.0000   0.0000   0.0000   0.0000
   3004    0.004450044128608    0.0  365.58    -549.154033651285    -549.154033651285    -548.987323076003    -548.982873031875   0.0000   0.0000   0.0000   0.0000
   3005    0.004457822208169    0.0  365.91    -549.154194886807    -549.154194886807    -548.987331087672    -548.982873265463   0.0000   0.0000   0.0000   0.0000
   3006    0.004463412329690    0.0  366.19    -549.154326699879    -549.154326699879    -548.987336869917    -5

     restart      :      2.48s CPU      2.09s WALL (       9 calls)
   3051    0.004278657496387    0.0  367.94    -549.154939039091    -549.154939039091    -548.987151344694    -548.982872687198   0.0000   0.0000   0.0000   0.0000
   3052    0.004284794208933    0.0  367.80    -549.154883078022    -549.154883078022    -548.987157074248    -548.982872280039   0.0000   0.0000   0.0000   0.0000
   3053    0.004290808815940    0.0  367.64    -549.154815770334    -549.154815770334    -548.987162665926    -548.982871857110   0.0000   0.0000   0.0000   0.0000
   3054    0.004297838233807    0.0  367.47    -549.154743372186    -549.154743372186    -548.987169292325    -548.982871454091   0.0000   0.0000   0.0000   0.0000
   3055    0.004306245678707    0.0  367.30    -549.154672149319    -549.154672149319    -548.987177293442    -548.982871047763   0.0000   0.0000   0.0000   0.0000
   3056    0.004315233087750    0.0  367.14    -549.154608153104    -549.154608153104    -548.987185878869    -5



                total energy =     -549.15277743901 Hartree a.u.
              kinetic energy =      396.81557 Hartree a.u.
        electrostatic energy =     -744.92006 Hartree a.u.
                         esr =        0.04738 Hartree a.u.
                       eself =      970.22763 Hartree a.u.
      pseudopotential energy =      -38.36665 Hartree a.u.
  n-l pseudopotential energy =      -27.24280 Hartree a.u.
 exchange-correlation energy =     -135.43884 Hartree a.u.
           average potential =        0.00000 Hartree a.u.



   Eigenvalues (eV), kp =   1 , spin =  1

  -23.04  -22.99  -22.95  -22.78  -22.72  -22.57  -22.55  -22.48  -22.47  -22.41
  -22.39  -22.35  -22.28  -22.25  -22.23  -22.15  -22.07  -22.03  -22.01  -21.86
  -21.81  -21.75  -21.72  -21.67  -21.65  -21.56  -21.50  -21.47  -21.43  -21.31
  -21.30  -21.23  -11.08  -11.03  -11.01  -10.92  -10.80  -10.66  -10.57  -10.51
  -10.46  -10.33  -10.31  -10.25  -10.16  -10.12  -10.08  -10.05  -10.04   -9.97
   -9.95  

0


## *III.III* - READING OUTPUT FILES

In [7]:
def get_evp():
    return pd.read_csv(out_dir+'H2O.evp',  names=['', 'time (ps)', 'ekinc', 'T_ion (K)', 'etot', 'econs', 'econt'], 
                       index_col=0,  usecols=[0,1,2,4,5,7,8],  delim_whitespace=True,  comment='#')

def get_nstep():
    evp = get_evp()
    return evp.shape[0]+1

def get_time(units='ps'):
    units_dict = {'fs':1e3, 'ps':1, 'ns':1e-3, 'μs':1e-6, 'ms':1e-9}
    
    evp = get_evp()
    nstep = evp.shape[0]+1
    
    time = np.zeros(nstep)
    time[1:] = evp['time (ps)'].values * units_dict[units]
    return time

def read_positions():
    nstep = get_nstep()
    pos = np.zeros((nstep, N_H2O*3, 3))
    
    init_pos = read_init_pos()
    for j,poss in enumerate(init_pos.split('\n')[2:-1]):
        pos_tmp = poss.split()
        pos[0,j,:] = float(pos_tmp[1]), float(pos_tmp[2]), float(pos_tmp[3])
        
    with open(out_dir+'H2O.pos', 'r') as f:
        for i in range(1,nstep):
            f.readline() # discard the first line
            for j in range(3*N_H2O):
                pos_tmp = f.readline().split()
                pos[i,j,:] = float(pos_tmp[0])*a0, float(pos_tmp[1])*a0, float(pos_tmp[2])*a0
    return pos

def read_velocities():
    nstep = get_nstep()
    vel = np.zeros((nstep, N_H2O*3))

    with open(out_dir+'H2O.vel', 'r') as f:
        for i in range(1,nstep):
            f.readline() # discard the first line
            for j in range(N_H2O*3):
                vel_tmp = f.readline().split()
                vel[i,j] = abs(np.mean([float(vel_tmp[k]) for k in range(3)]))
    return vel
                
def read_forces():
    nstep = get_nstep()
    forces = np.zeros((nstep, N_H2O*3))

    with open(out_dir+'H2O.for', 'r') as f:
        for i in range(1,nstep):
            f.readline() # discard the first line
            for j in range(N_H2O*3):
                forces_tmp = f.readline().split()
                forces[i,j] = abs(np.mean([float(forces_tmp[k]) for k in range(3)]))
    return forces

def write_trajectory():
    init_pos = read_init_pos()
    pos = read_positions()
    
    with open(out_dir+'H2O.traj.xyz', 'w') as f:
        for i in range(len(pos)):
            f.write(init_pos.split('\n')[0] + '\n' + init_pos.split('\n')[1] + '\n') # xyz header line
            for j,(x,y,z) in enumerate(pos[i]):
                if j%3==0: 
                    f.write(f'  O           {x:9.6f}       {y:9.6f}       {z:9.6f}\n')
                else:
                    f.write(f'  H           {x:9.6f}       {y:9.6f}       {z:9.6f}\n')

The file `H2O.evp` contains some thermodynamics data:
- `ekinc` is the electrons fictitious kinetic energy. In this case it is zero because we are not running yet the Verlet algorithm, and there is no lagrangian yet that can be used to define an electronic classical (fake) kinetic energy.
- `T_ion` is the temperature of the ion, computed as $T = \frac{2}{3Nk_B}\sum_I\frac{m_Iv_I^2}{2}$.
- `etot` is the DFT (potential) energy of the system.
- `econs` is the potential energy plus the kinetic energy of the nuclei and it is a physically meaningful constant of motion in the limit of zero electronic fictitious mass.
- `econt` is the constant of motion of the CP Lagrangian [(Eq. 3)](#eq-cpla). If the time step $dt$ is small enough this will be up to a very good precision a constant, although it is not a physical quantity, since the "fake" kinetic energy defined in $\mathcal{L}$ has nothing to do with the quantum kinetic energy of the electrons.

In [ ]:
## THERMODYNAMICS
evp = get_evp()

time_units = 'fs'
time = get_time(time_units)

### PLOT
fig, ax = plt.subplots(1,2, figsize=(15,6), constrained_layout=True)

# temperature
ax[0].plot(time[1:], evp['T_ion (K)'], '.-')
ax[0].set(xlim=(0,time[1]+time[-1]), ylim=(0,None), xlabel=r'$t\ ({})$'.format(time_units), ylabel=r'$T\ (K)$')

# energy
ax[1].plot(time[1:], evp['etot'], '.-', label='etot')
ax[1].plot(time[1:], evp['econs'], '.-', label='econs')
ax[1].plot(time[1:], evp['econt'], '.-', label='econt')
ax[1].set(xlim=(0,time[1]+time[-1]), xlabel=r'$t\ ({})$'.format(time_units), ylabel=r'$E\ (a.u.)$')
ax[1].legend(fontsize=14, fancybox=True, shadow=True)

plt.show()

The files `H2O.pos`, `H2O.vel` and `H2O.for` contain, respectively, the positions, the velocities and the forces of every atom in the simulation box. The following code also produces a trajectory file (in "extended xyz" format [$^{[7]}$](#bib-xyz)) named `H2O.traj.xyz`.

In [ ]:
## Write the trajectory file in an "extended xyz" format.
write_trajectory() 

# Then, it can be visualized with some molecule viewers software as`ovito`, 'VMD' or 'Jmol'.
view_cmd = '$HOME/Programs/ovito-3.8.3/bin/ovito'
os.system(view_cmd+' Output/H2O.traj.xyz')

In [ ]:
## VELOCITIES AND FORCES
velocities = read_velocities()
forces = read_forces()

time_units = 'fs'
time = get_time(time_units)

## PLOT
fig, ax = plt.subplots(1, 2, figsize=(15,6), constrained_layout=True)

for i in range(N_H2O*3): ax[0].plot(time[1:], velocities[1:,i], 'D', ms=4., alpha=0.1, c='blue')
ax[0].set(xlim=(0,time[1]+time[-1]), ylim=(0,None), 
          xlabel=r'$t\ ({})$'.format(time_units), ylabel=r'$v\ (a.u.)$')
    
for i in range(N_H2O*3): ax[1].plot(time[1:], forces[1:,i], 'D', ms=4., alpha=0.1, c='red')
ax[1].plot(time[1:], np.mean(forces[1:,:], axis=1), '--', lw=1., c='red') # mean value of the forces>
ax[1].set(xlim=(0,time[1]+time[-1]), ylim=(2e-6,1), 
          xlabel=r'$t\ ({})$'.format(time_units), ylabel=r'$F\ (a.u.)$', yscale='log')

plt.show()

# *IV* - RESULTS

In [10]:
@njit()
def get_msd(pos):
    nstep = len(pos)
    
    msd = np.zeros((nstep, 2))
    for k in range(nstep):
        for i in range(N_H2O):
            msd[k,0] += np.linalg.norm(pos[k,3*i,:]-pos[0,3*i,:])**2/N_H2O

            msd[k,1] += np.linalg.norm(pos[k,3*i+1,:]-pos[0,3*i+1,:])**2/N_H2O/2
            msd[k,1] += np.linalg.norm(pos[k,3*i+2,:]-pos[0,3*i+2,:])**2/N_H2O/2
    return msd

@njit()
def count_bonds(pos, bonds, cutoff, nbins, p):
    if bonds == 'all':
        g = np.zeros((3*N_H2O, p**3*3*N_H2O))
        for i in range(3*N_H2O):
            x0, y0, z0 = pos[i,:]

            t = 0
            for j in range(3*N_H2O):
                for lx in range(0,p):
                    for ly in range(0,p):
                        for lz in range(0,p):
                            x, y, z = pos[j,:] + np.array([L*(lx-p//2), L*(ly-p//2), L*(lz-p//2)])
                            dist = abs(np.sqrt((x0-x)**2+(y0-y)**2+(z0-z)**2))

                            g[i,t] = dist if dist != 0 else np.nan
                            t += 1
    elif bonds == 'O-O':
        g = np.zeros((N_H2O, p**3*N_H2O))
        for i in range(N_H2O):
            x0, y0, z0 = pos[3*i,:]

            t = 0
            for j in range(N_H2O):
                for lx in range(0,p):
                    for ly in range(0,p):
                        for lz in range(0,p):
                            x, y, z = pos[3*j,:] + np.array([L*(lx-p//2), L*(ly-p//2), L*(lz-p//2)])
                            dist = abs(np.sqrt((x0-x)**2+(y0-y)**2+(z0-z)**2))

                            g[i,t] = dist if dist != 0 else np.nan
                            t += 1
    elif bonds == 'O-H':
        g = np.zeros((N_H2O, p**3*2*N_H2O))
        for i in range(N_H2O):
            x0, y0, z0 = pos[3*i,:]

            t = 0
            for j in range(N_H2O):
                for lx in range(0,p):
                    for ly in range(0,p):
                        for lz in range(0,p):
                            x, y, z = pos[3*j+1,:] + np.array([L*(lx-p//2), L*(ly-p//2), L*(lz-p//2)])
                            dist = abs(np.sqrt((x0-x)**2+(y0-y)**2+(z0-z)**2))
                            g[i,t] = dist if dist != 0 else np.nan
                            t += 1

                            x, y, z = pos[3*j+2,:] + np.array([L*(lx-p//2), L*(ly-p//2), L*(lz-p//2)])
                            dist = abs(np.sqrt((x0-x)**2+(y0-y)**2+(z0-z)**2))
                            g[i,t] = dist if dist != 0 else np.nan
                            t += 1
    elif bonds == 'H-H':
        g = np.zeros((2*N_H2O, p**3*2*N_H2O))
        for i in range(N_H2O):
            x0, y0, z0 = pos[3*i+1,:]

            t = 0
            for j in range(N_H2O):
                for lx in range(0,p):
                    for ly in range(0,p):
                        for lz in range(0,p):
                            x, y, z = pos[3*j+1,:] + np.array([L*(lx-p//2), L*(ly-p//2), L*(lz-p//2)])
                            dist = abs(np.sqrt((x0-x)**2+(y0-y)**2+(z0-z)**2))
                            g[2*i,t] = dist if dist != 0 else np.nan
                            t += 1

                            x, y, z = pos[3*j+2,:] + np.array([L*(lx-p//2), L*(ly-p//2), L*(lz-p//2)])
                            dist = abs(np.sqrt((x0-x)**2+(y0-y)**2+(z0-z)**2))
                            g[2*i,t] = dist if dist != 0 else np.nan
                            t += 1

            x0, y0, z0 = pos[3*i+2,:]

            t = 0
            for j in range(N_H2O):
                for lx in range(0,p):
                    for ly in range(0,p):
                        for lz in range(0,p):
                            x, y, z = pos[3*j+1,:] + np.array([L*(lx-p//2), L*(ly-p//2), L*(lz-p//2)])
                            dist = abs(np.sqrt((x0-x)**2+(y0-y)**2+(z0-z)**2))
                            g[2*i+1,t] = dist if dist != 0 else np.nan
                            t += 1

                            x, y, z = pos[3*j+2,:] + np.array([L*(lx-p//2), L*(ly-p//2), L*(lz-p//2)])
                            dist = abs(np.sqrt((x0-x)**2+(y0-y)**2+(z0-z)**2))
                            g[2*i+1,t] = dist if dist != 0 else np.nan
                            t += 1
    return g
    
def get_gofr(pos, bonds='all', cutoff=10, nbins=200, p=3):
    g = count_bonds(pos, bonds, cutoff, nbins, p)
    
    gofr, bins = np.histogram(g, bins=nbins, range=(0.,cutoff))
    bins = bins[1:]
    if bonds == 'all': norm = 9
    if bonds == 'O-O': norm = 1
    if bonds == 'O-H': norm = 2
    if bonds == 'H-H': norm = 4
        
    gofr = gofr/(norm*N_H2O**2/L**3*4*np.pi*bins**2*cutoff/nbins)
    return bins, gofr

### MEAN-SQUARED DISPLACEMENT

In [11]:
time_units = 'fs'
time = get_time(time_units)

msd = get_msd(read_positions())

fig, ax = plt.subplots(figsize=(8,6), constrained_layout=True)

ax.plot(time, msd[:,0], '.-', c='red', label='O')
ax.plot(time, msd[:,1], '.-', c='blue', label='H')

ax.legend(fontsize=12, fancybox=True, shadow=True, loc='upper left')
ax.set(xlim=(0,time[-1]), ylim=(0,None), xlabel=r'$t\ ({})$'.format(time_units), ylabel=r'$msd\ (Å)$')

plt.show()

### PAIR CORRELATION FUNCTION

In [12]:
nbins = 200
cutoff = L
p = 5
step = 0

pos = read_positions()
bins    , gofr     = get_gofr(pos[step], bonds='all', cutoff=cutoff, nbins=nbins, p=p)
bins_O_O, gofr_O_O = get_gofr(pos[step], bonds='O-O', cutoff=cutoff, nbins=nbins, p=p)
bins_O_H, gofr_O_H = get_gofr(pos[step], bonds='O-H', cutoff=cutoff, nbins=nbins, p=p)
bins_H_H, gofr_H_H = get_gofr(pos[step], bonds='H-H', cutoff=cutoff, nbins=nbins, p=p)

fig, ax = plt.subplots(figsize=(15,6), constrained_layout=True)

ax.plot(bins, gofr, c='black')

ax_ins = ax.inset_axes([0.45,0.35,0.55,0.65])
ax_ins_O_O = ax_ins.inset_axes([0., 0., 1., 1/3])
ax_ins_O_O.plot(bins_O_O, gofr_O_O, c='red')
ax_ins_O_O.text(0.7, 0.7, 'O-O', c='red', transform=ax_ins_O_O.transAxes, bbox=dict(facecolor='none'))
ax_ins_O_O.set(xlim=(0,cutoff), ylim=(0, np.max(gofr_O_O)*1.2), xticklabels=[], 
               yticks=[int(np.max(gofr_O_O)/2), int(np.max(gofr_O_O))])

ax_ins_H_H = ax_ins.inset_axes([0., 1/3, 1., 1/3])
ax_ins_H_H.plot(bins_H_H, gofr_H_H, c='blue')
ax_ins_H_H.text(0.7, 0.7, 'H-H', c='blue', transform=ax_ins_H_H.transAxes, bbox=dict(facecolor='none'))
ax_ins_H_H.set(xlim=(0,cutoff), ylim=(0, np.max(gofr_H_H)*1.2), xticklabels=[], 
               yticks=[int(np.max(gofr_H_H)/2), int(np.max(gofr_H_H))])

ax_ins_O_H = ax_ins.inset_axes([0., 2/3, 1., 1/3])
ax_ins_O_H.plot(bins_O_H, gofr_O_H, c='green')
ax_ins_O_H.text(0.7, 0.7, 'O-H', c='green', transform=ax_ins_O_H.transAxes, bbox=dict(facecolor='none'))
ax_ins_O_H.set(xlim=(0,cutoff), ylim=(0, np.max(gofr_O_H)*1.2), xticklabels=[], 
               yticks=[int(np.max(gofr_O_H)/2), int(np.max(gofr_O_H))])

ax.set(xlim=(0,cutoff), ylim=(0, None), xlabel='$r\ (Å)$', ylabel='$g(r)$')

ax_ins.set(xlim=(0,cutoff), ylim=(0, None), xlabel='$r\ (Å)$', yticks=[])
ax_ins.set_ylabel(ylabel='$g(r)$', labelpad=30.)

plt.show()

/home/acuoghi/anaconda3/envs/intelpython/lib/python3.10/site-packages/numpy/lib/histograms.py:837: RuntimeWarning: invalid value encountered in greater_equal
  keep = (tmp_a >= first_edge)
/home/acuoghi/anaconda3/envs/intelpython/lib/python3.10/site-packages/numpy/lib/histograms.py:838: RuntimeWarning: invalid value encountered in less_equal
  keep &= (tmp_a <= last_edge)
/home/acuoghi/anaconda3/envs/intelpython/lib/python3.10/site-packages/numpy/lib/histograms.py:837: RuntimeWarning: invalid value encountered in greater_equal
  keep = (tmp_a >= first_edge)
/home/acuoghi/anaconda3/envs/intelpython/lib/python3.10/site-packages/numpy/lib/histograms.py:838: RuntimeWarning: invalid value encountered in less_equal
  keep &= (tmp_a <= last_edge)
/home/acuoghi/anaconda3/envs/intelpython/lib/python3.10/site-packages/numpy/lib/histograms.py:837: RuntimeWarning: invalid value encountered in greater_equal
  keep = (tmp_a >= first_edge)
/home/acuoghi/anaconda3/envs/intelpython/lib/python3.10/site-

# *V* - CONCLUSIONS

---
# APPENDIX

# BIBLIOGRAPHY
1. *L. Verlet*, [Phys. Rev. **159**, 98 (1967)](https://doi.org/10.1103/PhysRev.159.98).<a id='bib-ver'></a>
2. *W. G. Hoover*, [Phys. Rev. A **31**, 1695 (1985)](https://doi.org/10.1103/PhysRevA.31.1695).<a id='bib-hoo'></a>
3. *R. Car and M. Parrinello*, [Phys. Rev. Lett. **55**, 2471 (1985)](https://doi.org/10.1103/PhysRevLett.55.2471).<a id='bib-cpmd'></a>
4. *P. Giannozzi et al.*, [J. Phys.: Condens. Matter 21, 395502 (2009)](https://doi.org/10.1088/0953-8984/21/39/395502)<a id='bib-qe1'></a>
5. *P. Giannozzi et al.*, [J. Phys.: Condens. Matter 29, 465901 (2017)](https://doi.org/10.1088/1361-648X/aa8f79)<a id='bib-qe2'></a>
6. *L. Martínez, R. Andrade, E. G. Birgin and J. M. Martínez*, [J. Comput. Chem. 30(13):2157-2164 (2009)](https://doi.org/10.1002/jcc.21224).<a id='bib-pack'></a>
7. XYZ file format, [Wikipedia page](https://en.wikipedia.org/wiki/XYZ_file_format).<a id='bib-xyz'></a>

In [17]:
cp_run('cp.water.nve-restart.in', n_mpi=2, n_omp=2)


     Program CP v.7.2 starts on 21Apr2023 at 18:16:41 

     This program is part of the open-source Quantum ESPRESSO suite
     for quantum simulation of materials; please cite
         "P. Giannozzi et al., J. Phys.:Condens. Matter 21 395502 (2009);
         "P. Giannozzi et al., J. Phys.:Condens. Matter 29 465901 (2017);
         "P. Giannozzi et al., J. Chem. Phys. 152 154105 (2020);
          URL http://www.quantum-espresso.org", 
     in publications or presentations arising from this work. More details at
     http://www.quantum-espresso.org/quote

     Parallel version (MPI & OpenMP), running on       4 processor cores
     Number of MPI processes:                 2
     Threads/MPI process:                     2

     MPI processes distributed on     1 nodes
     R & G space division:  proc/nbgrp/npool/nimage =       2
     795 MiB available memory on the printing compute node when the environment starts
 
     Reading input from /home/acuoghi/Programmazione/LQSM/Water/QE-wat


   formf: eself=   970.22763
   formf:     vps(g=0)=  -0.0001855     rhops(g=0)=  -0.0008915
   formf:     vps(g=0)=  -0.0001841     rhops(g=0)=  -0.0008854
   formf:     vps(g=0)=  -0.0001841     rhops(g=0)=  -0.0008854
   formf:     vps(g=0)=  -0.0001827     rhops(g=0)=  -0.0008793
   formf:     vps(g=0)=  -0.0001827     rhops(g=0)=  -0.0008793
   formf: sum_g vps(g)=  -0.9579407 sum_g rhops(g)=  -4.3105308
   formf:     vps(g=0)=  -0.0000904     rhops(g=0)=  -0.0001486
   formf:     vps(g=0)=  -0.0000900     rhops(g=0)=  -0.0001476
   formf:     vps(g=0)=  -0.0000900     rhops(g=0)=  -0.0001476
   formf:     vps(g=0)=  -0.0000896     rhops(g=0)=  -0.0001465
   formf:     vps(g=0)=  -0.0000896     rhops(g=0)=  -0.0001465
   formf: sum_g vps(g)=  -1.1465899 sum_g rhops(g)=  -0.7184218
   Delta V(G=0):   0.029852Ry,    0.812310eV
 
  nfi     ekinc              temph  tempp     etot                 enthal               econs                econt              vnhh    xnhh0   vnhp    xnh

   9145    0.005172946136982    0.0  411.87    -549.175872153894    -549.175872153894    -548.988050197940    -548.982877251803   0.0000   0.0000   0.0000   0.0000
   9146    0.005165837013809    0.0  411.88    -549.175870686586    -549.175870686586    -548.988042921049    -548.982877084035   0.0000   0.0000   0.0000   0.0000
   9147    0.005157555282858    0.0  411.76    -549.175804330123    -549.175804330123    -548.988034399953    -548.982876844670   0.0000   0.0000   0.0000   0.0000
   9148    0.005148430327312    0.0  411.50    -549.175676888327    -549.175676888327    -548.988024962486    -548.982876532159   0.0000   0.0000   0.0000   0.0000
   9149    0.005138824071759    0.0  411.12    -549.175493349106    -549.175493349106    -548.988014978687    -548.982876154615   0.0000   0.0000   0.0000   0.0000
   9150    0.005128841815180    0.0  410.63    -549.175259770490    -549.175259770490    -548.988004550986    -548.982875709171   0.0000   0.0000   0.0000   0.0000

   writing rest

   9194    0.005040525361919    0.0  391.97    -549.166660834957    -549.166660834957    -548.987913687356    -548.982873161994   0.0000   0.0000   0.0000   0.0000
   9195    0.005024002483367    0.0  390.10    -549.165788605747    -549.165788605747    -548.987896171330    -548.982872168847   0.0000   0.0000   0.0000   0.0000
   9196    0.005006726256419    0.0  388.27    -549.164936474785    -549.164936474785    -548.987877856951    -548.982871130695   0.0000   0.0000   0.0000   0.0000
   9197    0.004989227986014    0.0  386.52    -549.164121094757    -549.164121094757    -548.987859292009    -548.982870064023   0.0000   0.0000   0.0000   0.0000
   9198    0.004972047958852    0.0  384.89    -549.163359468661    -549.163359468661    -548.987841040358    -548.982868992399   0.0000   0.0000   0.0000   0.0000
   9199    0.004955610281000    0.0  383.41    -549.162668508841    -549.162668508841    -548.987823570959    -548.982867960678   0.0000   0.0000   0.0000   0.0000

 * Physical Qua

   9201    0.004925887552734    0.0  381.06    -549.161562167420    -549.161562167420    -548.987792065069    -548.982866177516   0.0000   0.0000   0.0000   0.0000
   9202    0.004913192488305    0.0  380.24    -549.161174774991    -549.161174774991    -548.987778685604    -548.982865493116   0.0000   0.0000   0.0000   0.0000
   9203    0.004902682672484    0.0  379.69    -549.160912953158    -549.160912953158    -548.987767677264    -548.982864994592   0.0000   0.0000   0.0000   0.0000
   9204    0.004894821906491    0.0  379.42    -549.160784494154    -549.160784494154    -548.987759532868    -548.982864710962   0.0000   0.0000   0.0000   0.0000
   9205    0.004889592673695    0.0  379.46    -549.160793864267    -549.160793864267    -548.987754240895    -548.982864648221   0.0000   0.0000   0.0000   0.0000
   9206    0.004886623590076    0.0  379.79    -549.160942005281    -549.160942005281    -548.987751444550    -548.982864820960   0.0000   0.0000   0.0000   0.0000
   9207    0.004

...

   9801    0.005287377969035    0.0  435.07    -549.186559339670    -549.186559339670    -548.988160011887    -548.982872633918   0.0000   0.0000   0.0000   0.0000
   9802    0.005274886643363    0.0  433.56    -549.185859331416    -549.185859331416    -548.988146961665    -548.982872075022   0.0000   0.0000   0.0000   0.0000
   9803    0.005262796162578    0.0  432.13    -549.185194022536    -549.185194022536    -548.988134453938    -548.982871657775   0.0000   0.0000   0.0000   0.0000
   9804    0.005251064149684    0.0  430.79    -549.184570033904    -549.184570033904    -548.988122456625    -548.982871392476   0.0000   0.0000   0.0000   0.0000
   9805    0.005239907053768    0.0  429.54    -549.183991643300    -549.183991643300    -548.988111190748    -548.982871283695   0.0000   0.0000   0.0000   0.0000
   9806    0.005229555301319    0.0  428.40    -549.183460723480    -549.183460723480    -548.988100884545    -548.982871329244   0.0000   0.0000   0.0000   0.0000
   9807    0.005

   9851    0.004905189075558    0.0  417.19    -549.178023015823    -549.178023015823    -548.987774595619    -548.982869406544   0.0000   0.0000   0.0000   0.0000
   9852    0.004905835931674    0.0  417.17    -549.178013308974    -549.178013308974    -548.987774524562    -548.982868688630   0.0000   0.0000   0.0000   0.0000
   9853    0.004908692574856    0.0  417.33    -549.178086734542    -549.178086734542    -548.987776849285    -548.982868156710   0.0000   0.0000   0.0000   0.0000
   9854    0.004914029119840    0.0  417.68    -549.178251556931    -549.178251556931    -548.987781839217    -548.982867810097   0.0000   0.0000   0.0000   0.0000
   9855    0.004921773711384    0.0  418.23    -549.178513123745    -549.178513123745    -548.987789432099    -548.982867658388   0.0000   0.0000   0.0000   0.0000
   9856    0.004931623764144    0.0  419.00    -549.178873655283    -549.178873655283    -548.987799343033    -548.982867719268   0.0000   0.0000   0.0000   0.0000
   9857    0.004

   9901    0.005018920356413    0.0  416.54    -549.177843920543    -549.177843920543    -548.987893774164    -548.982874853808   0.0000   0.0000   0.0000   0.0000
   9902    0.005011602441176    0.0  415.90    -549.177546086854    -549.177546086854    -548.987886422716    -548.982874820275   0.0000   0.0000   0.0000   0.0000
   9903    0.005003747683815    0.0  415.24    -549.177237358603    -549.177237358603    -548.987878440115    -548.982874692431   0.0000   0.0000   0.0000   0.0000
   9904    0.004995648825532    0.0  414.56    -549.176919560819    -549.176919560819    -548.987870160272    -548.982874511447   0.0000   0.0000   0.0000   0.0000
   9905    0.004987437018458    0.0  413.87    -549.176595250015    -549.176595250015    -548.987861743297    -548.982874306278   0.0000   0.0000   0.0000   0.0000
   9906    0.004979005254686    0.0  413.17    -549.176267291628    -549.176267291628    -548.987853126932    -548.982874121678   0.0000   0.0000   0.0000   0.0000
   9907    0.004

   9951    0.004862984625279    0.0  415.08    -549.177027895629    -549.177027895629    -548.987742297399    -548.982879312773   0.0000   0.0000   0.0000   0.0000
   9952    0.004869035585814    0.0  415.96    -549.177436644957    -549.177436644957    -548.987748751573    -548.982879715987   0.0000   0.0000   0.0000   0.0000
   9953    0.004873461808613    0.0  416.66    -549.177757283864    -549.177757283864    -548.987753439654    -548.982879977846   0.0000   0.0000   0.0000   0.0000
   9954    0.004876150099453    0.0  417.15    -549.177986067391    -549.177986067391    -548.987756242532    -548.982880092432   0.0000   0.0000   0.0000   0.0000
   9955    0.004877052918717    0.0  417.45    -549.178121570796    -549.178121570796    -548.987757109293    -548.982880056374   0.0000   0.0000   0.0000   0.0000
   9956    0.004876150248035    0.0  417.54    -549.178164743193    -549.178164743193    -548.987756024570    -548.982879874322   0.0000   0.0000   0.0000   0.0000
   9957    0.004

0
0


- From step 1 to step 150 -> BOMD
- From step 151 to step 2600 -> Nosè
- Step 2601 -> CG
- From step 2602 to step 10000 -> CPMD